In [8]:
from datetime import datetime
from operator import itemgetter
from os import path

from astropy.table import Table
from skynet.api.v20 import errors
from skynet.api.v20.methods import download, groups, obs

# for turning into command line program:
    # -b begtime
    # -e endtime
    # -d data_directory [Data]
    # -o
    # -i


In [2]:
# define parameters to control the code below

data_directory = "Data"
start_time = "2022-09-06T00:00:00" # only download files after this time
# start_time = "2024-08-15 00:00:00" # only download files after this time
observation_criteria = {"groupId": groups.get("Majewski Research Group")["id"]}
exposure_criteria = observation_criteria

# scope_id = scopes.query(include="*", **{"name": "RRRT"})[0]["id"]

observation_criteria

{'groupId': 816}

In [3]:
# retrieve list of observations happening after start_time and meeting observation_criteria
# write to csv just in case it's useful

observation_fields = {
    "id": "Observation ID",
    "state": "Observation state",
    "priority": "Priority",
    "name": "Target Name",
    "user.username": "Username",
    "group.name": "Group",
    "raHours": "RA",
    "decDegs": "Dec",
    "currentTelescope.name": "Telescope",
    "teleOwner.name": "Scope Owner",
    "timeIn": "Submitted",
}

observation_results = obs.query(include=",".join(observation_fields.keys()), after=start_time, **observation_criteria)

observations = Table()
for field, col_name in observation_fields.items():
    field_parts = field.split(".")
    match len(field_parts):
        case 1:
            observations[col_name] = list(map(itemgetter(field), observation_results))
        case 2:
            field_dicts = list(map(itemgetter(field_parts[0]), observation_results))
            observations[col_name] = [field_dict[field_parts[1]] if field_dict is not None else "" for field_dict in field_dicts]
observations.write("skynet_observations.csv", overwrite=True)

observations

Observation ID,Observation state,Priority,Target Name,Username,Group,RA,Dec,Telescope,Scope Owner,Submitted
int32,str9,int32,str29,str9,str23,float64,float64,str4,str3,str19
10067782,canceled,7,TIC 269811101,jdavidson,Majewski Research Group,23.392252777777777,60.886916666666664,RRRT,UVA,2023-09-19T03:41:44
10067789,canceled,7,TIC 375325607,jdavidson,Majewski Research Group,21.052872222222224,55.46931388888889,,UVA,2023-09-19T03:58:54
10277972,canceled,1,TIC 239872462 Ab,ef4taq,Majewski Research Group,5.8342777777777775,34.41759722222222,,UVA,2023-11-09T22:20:21
10279223,completed,1,TIC 239872462 Ab,ef4taq,Majewski Research Group,5.8342777777777775,34.41759722222222,RRRT,UVA,2023-11-10T16:11:01
10286988,completed,5,TIC 239872462 Ab,ef4taq,Majewski Research Group,5.8342775,34.41759444444444,RRRT,UVA,2023-11-11T18:49:29
10287062,completed,5,TIC 239872462 Ab,ef4taq,Majewski Research Group,5.834277222222222,34.41759166666667,RRRT,UVA,2023-11-12T04:57:35
10287063,completed,5,TIC 239872462 Ab,ef4taq,Majewski Research Group,5.8342769444444444,34.417588888888886,RRRT,UVA,2023-11-12T05:04:17
10287064,completed,5,TIC 239872462 Ab,ef4taq,Majewski Research Group,5.8342769444444444,34.417586111111106,RRRT,UVA,2023-11-12T05:27:06
10291478,completed,5,TIC 239872462 Ab,ef4taq,Majewski Research Group,5.8342769444444444,34.41758333333333,RRRT,UVA,2023-11-13T01:53:33


In [4]:
# retrieve list of images happening after start_time and meeting exposure_criteria
# write to csv just in case it's useful

exposure_fields = {
    "exps.obsId": "Observation ID",
    "exps.expNum": "Exposure Num",
    "exps.imageId": "Image ID",
    "exps.obs.name": "Name",
    "exps.type": "Type",
    "exps.filterName": "Filter Name",
    "exps.timeTaken": "Time Taken",
    "exps.state": "Image State",
    "exps.expLengthUsed": "Exposure",
    "exps.binningUsed": "Binning",
    "exps.gain": "Gain",
    "exps.camera.name": "Camera",
}

exposure_results = obs.query(include=",".join(exposure_fields.keys()), after=start_time, **exposure_criteria)

exp_structs = [image for result in exposure_results for image in result["exps"]]
images = Table()
for field, col_name in exposure_fields.items():
    field_parts = field.split(".")
    match len(field_parts):
        case 2:
            images[col_name] = list(map(itemgetter(field_parts[1]), exp_structs))
        case 3:
            field_dicts = list(map(itemgetter(field_parts[1]), exp_structs))
            images[col_name] = [field_dict[field_parts[2]] if field_dict is not None else "" for field_dict in field_dicts]
images.write("skynet_images.csv", overwrite=True)

images

Observation ID,Exposure Num,Image ID,Name,Type,Filter Name,Time Taken,Image State,Exposure,Binning,Gain,Camera
int32,int32,str9,str29,str5,str6,object,str8,object,int32,object,str40
10067782,0,r38972029,TIC 269811101,light,V,2023-09-19T03:42:09,archived,60.0,2,1.33183,UVA SBIG STX-16803 with FLI Filter Wheel
10067782,1,r38972030,TIC 269811101,light,V,2023-09-19T03:43:18,archived,60.0,2,1.33183,UVA SBIG STX-16803 with FLI Filter Wheel
10067782,2,r38972031,TIC 269811101,light,V,2023-09-19T03:44:26,archived,60.0,2,1.33183,UVA SBIG STX-16803 with FLI Filter Wheel
10067782,3,r38972032,TIC 269811101,light,V,2023-09-19T03:45:34,archived,60.0,2,1.33183,UVA SBIG STX-16803 with FLI Filter Wheel
10067782,4,r38972033,TIC 269811101,light,V,2023-09-19T03:46:42,archived,60.0,2,1.33183,UVA SBIG STX-16803 with FLI Filter Wheel
10067782,5,r38972034,TIC 269811101,light,V,2023-09-19T03:47:50,archived,60.0,2,1.33183,UVA SBIG STX-16803 with FLI Filter Wheel
10067782,6,r38972035,TIC 269811101,light,V,2023-09-19T03:48:57,archived,60.0,2,1.33183,UVA SBIG STX-16803 with FLI Filter Wheel
10067782,7,r38972036,TIC 269811101,light,V,2023-09-19T03:50:05,archived,60.0,2,1.33183,UVA SBIG STX-16803 with FLI Filter Wheel
10067782,8,r38972037,TIC 269811101,light,V,2023-09-19T03:51:12,archived,60.0,2,1.33183,UVA SBIG STX-16803 with FLI Filter Wheel


In [6]:
# do the actual download of the identified files
# skip files that already exist

def download_file_part(data_directory, obs_id, target, num_parts, part_num):
    if num_parts > 1:
        filename = f"{data_directory}/{obs_id}-{target}_{part_num}.zip"
    else:
        filename = f"{data_directory}/{obs_id}-{target}.zip"
    if path.isfile(filename):
        print(f"  File {filename} already downloaded")
        return
    print(f"  Downloading part {part_num}...", end=" ")
    _, data = download.fits(**{"obs": obs_id}, total_parts=num_parts, part=part_num)
    filesize = len(data)
    duration = datetime.now() - start
    if filesize > 0:
        with open(filename, "wb") as f:
            f.write(data)
        print(f"got {filesize/1024/1024:7.1f}MB at {filesize / duration.seconds * 8 / 1024 / 1024:5.2f}Mbps saved to {filename}")

for observation in observations:
    obs_id = observation[observation_fields["id"]]
    target = observation[observation_fields["name"]].strip().replace(" ", "_")
    print(f"Downloading observation {obs_id:10d}")
    start = datetime.now()
    num_images = len(images[images["Observation ID"] == obs_id])
    num_parts = num_images // 1000 + 1
    for part_num in range(1, num_parts + 1):
        try:
            download_file_part(data_directory, obs_id, target, num_parts, part_num)
        except errors.BadRequestError:
            print(f"  skipped part {part_num}- got bad request")
            continue
print("---Downloads Complete---")

  File Data/10067782-TIC_269811101.zip already downloaded
  File Data/10067789-TIC_375325607.zip already downloaded
  File Data/10279223-TIC_239872462_Ab.zip already downloaded
  File Data/10286988-TIC_239872462_Ab.zip already downloaded
  File Data/10287062-TIC_239872462_Ab.zip already downloaded
  File Data/10287063-TIC_239872462_Ab.zip already downloaded
  File Data/10287064-TIC_239872462_Ab.zip already downloaded
  File Data/10291478-TIC_239872462_Ab.zip already downloaded
  File Data/10291510-TIC_239872462_SHIFT_111323UTC.zip already downloaded
  File Data/10291512-TIC_239872462_SHIFT_111323UTC.zip already downloaded
  File Data/10291513-TIC_239872462_SHIFT_111323UTC_1.zip already downloaded
  File Data/10291513-TIC_239872462_SHIFT_111323UTC_2.zip already downloaded
  File Data/10291513-TIC_239872462_SHIFT_111323UTC_3.zip already downloaded
  File Data/10291513-TIC_239872462_SHIFT_111323UTC_4.zip already downloaded
  File Data/10349683-TIC_4707_Cal.zip already downloaded
  File Da

ChunkedEncodingError: ('Connection broken: IncompleteRead(178 bytes read, 7822 more expected)', IncompleteRead(178 bytes read, 7822 more expected))

In [28]:
num_images = 1008
num_parts = num_images // 1000 + 1
list(range(1, num_parts+1))

[1, 2]